In [1]:
from tensorscaling import scale, unit_tensor, random_tensor, marginal
import numpy as np

# Tensor scaling

Scale 3x3x3 unit tensor to certain non-uniform marginals:

In [2]:
shape = [3, 3, 3]
targets = [(.5, .25, .25), (.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-4)
res

Result(success=True, iterations=64, max_dist=9.352462536334517e-05, ..., log_cap=-0.2736134077828343)

We can also access the scaling matrices and the final scaled state:

In [3]:
print(res.gs[0], "\n")
print(res.gs[1], "\n")
print(res.gs[2])

[[1.42874503+0.j         0.        +0.j         0.        +0.j        ]
 [0.22178887-0.01689832j 0.65844754+0.j         0.        +0.j        ]
 [0.04983856-0.37778883j 0.23384047+0.08091908j 0.96291855+0.j        ]] 

[[ 1.29235203+0.j          0.        +0.j          0.        +0.j        ]
 [ 0.41675364-0.61342456j  0.77386175+0.j          0.        +0.j        ]
 [ 0.33480015+0.68620936j -0.08917634+0.03808364j  1.05286993+0.j        ]] 

[[ 1.59561467+0.j          0.        +0.j          0.        +0.j        ]
 [-0.29781036-0.00479159j  0.6851713 +0.j          0.        +0.j        ]
 [ 0.05563831+0.16558015j  0.00333096-0.08479587j  0.4340156 +0.j        ]]


Let's now check that the W tensor *cannot* be scaled to uniform marginals:

In [4]:
shape = [2, 2, 2, 2]
W = np.zeros(shape)
W[1, 0, 0, 0] = W[0, 1, 0, 0] = W[0, 0, 1, 0] = W[0, 0, 0, 1] = .5
targets = [(.5, .5)] * 4

scale(W, targets, eps=1e-4, max_iterations=1000)

Result(success=False, iterations=1000, max_dist=0.2682023390104677, ..., log_cap=None)

To see more clearly what is going on, we can set the `verbose` flag:

In [5]:
res = scale(W, targets, eps=1e-4, max_iterations=10, verbose=True)

scaling tensor of shape (2, 2, 2, 2) and type float64
target spectra:
  0: (0.5, 0.5)
  1: (0.5, 0.5)
  2: (0.5, 0.5)
  3: (0.5, 0.5)
#000: max_dist = 0.35355339 @ sys = 1
#001: max_dist = 0.47140452 @ sys = 0
#002: max_dist = 0.56568542 @ sys = 2
#003: max_dist = 0.62853936 @ sys = 3
#004: max_dist = 0.58232323 @ sys = 1
#005: max_dist = 0.59305730 @ sys = 0
#006: max_dist = 0.59545834 @ sys = 2
#007: max_dist = 0.59262283 @ sys = 3
#008: max_dist = 0.59353004 @ sys = 1
#009: max_dist = 0.59357133 @ sys = 0
#010: max_dist = 0.59340661 @ sys = 2
did not converge!


We see that at each point in the algorithm, one of the marginals has Frobenius distance $\approx 0.59$ to being uniform. Indeed, we know that the entanglement polytope of the W tensor does not include the point corresponding to uniform marginals -- see [here](https://www.entanglement-polytopes.org/four_qubits) for an interactive visualization!

# Tuples of matrices and tensors

We can just as well only prescribe the desired spectra for subsystems.
Note that prescribing two out of three marginals amounts to *operator scaling*.

In [6]:
shape = [3, 3, 3]
targets = [(.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-6)
res

Result(success=True, iterations=47, max_dist=6.418032239360306e-07, ..., log_cap=-0.1986207824658252)

Indeed, the last two marginals are as prescribed, while the first marginal is arbitrary.

In [7]:
print(marginal(res.psi, 0).round(5), "\n")
print(marginal(res.psi, 1).round(5), "\n")
print(marginal(res.psi, 2).round(5))

[[0.3105 +0.j      0.00904+0.00713j 0.00762+0.00201j]
 [0.00904-0.00713j 0.35711+0.j      0.02813-0.01163j]
 [0.00762-0.00201j 0.02813+0.01163j 0.33238+0.j     ]] 

[[ 0.4+0.j  0. -0.j -0. -0.j]
 [ 0. +0.j  0.3+0.j  0. +0.j]
 [-0. +0.j  0. -0.j  0.3+0.j]] 

[[ 0.7+0.j -0. +0.j -0. +0.j]
 [-0. -0.j  0.2+0.j -0. -0.j]
 [-0. -0.j -0. +0.j  0.1+0.j]]
